In [4]:
from PIL import Image 
from IPython.display import display 
import numpy as np
import random
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import pymysql
from PIL import Image 
from datetime import datetime, timedelta, timezone
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

# Summary



[Read individuation table](#Read-individuation-table)

[Check activation table](#Check-activation-table)

[Add weights for matching](#Add-weights-for-matching)

[Run matching](#Run-matching)

[Confirm matches to send](#Confirm-matches-to-send)



Items that are not part of regular run:

[Delete records in match table](#Delete-records-in-match-table)

[Reintroduce review 'not found' (after a while)](#Reintroduce-review-'not-found'-(after-a-while))



----------------------------------

## Read individuation table

Read full individuation file

-> TO DO: adjust table from soul_form_beta to soul_form before mint

In [5]:
df = pd.DataFrame()

In [6]:
## SQL connection

conn=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor = conn.cursor()


## Get individuation records

df=pd.read_sql_query("SELECT * FROM XXX ",con=conn)
df = df.set_index('id')

## Close connection

conn.close()

In [7]:
df

,q_1,q_2,q_3,q_4,q_5,address,nonce,getID,TokenID,createdAt,Token
id,,,,,,,,,,,
1,@shaman4646,intuition,team_player,"enthusiast,optimistic,like_water,director,conn...","enthusiast,optimistic,like_water,director,conn...",,beta_test,1,47,2022-10-18 21:32:04,Genesis
2,@Koa_GM,imagination,founder,"empathetic,adventurer,like_water,artist,still,...","empathetic,adventurer,like_water,artist,still,...",,beta_test,1,46,2022-10-18 21:32:04,Genesis
3,@Lu_rolu,intuition,idealist,"traditionalist,controller,ingenious,enthusiast...","traditionalist,controller,ingenious,enthusiast...",,beta_test,1,45,2022-10-18 21:32:04,Genesis
4,NFTGuru2,intellect,mentor,"bonvivant,achiever,guided,adventurer,connector...","bonvivant,achiever,guided,adventurer,connector...",,beta_test,1,44,2022-10-18 21:32:04,Genesis
5,lima_eth,intellect,mentor,"bonvivant,like_water,champion,ally,moderator,m...","bonvivant,like_water,champion,ally,moderator,m...",,beta_test,1,43,2022-10-18 21:32:04,Genesis
6,@rp_rolu,imagination,visionary,"ally,artist,shapeshifter,connector,empathetic,...","ally,artist,shapeshifter,connector,empathetic,...",,beta_test,1,42,2022-10-18 21:32:04,Genesis
7,Vissyarts,intellect,explorer,"optimistic,shapeshifter,seeker,realist,like_wa...","optimistic,shapeshifter,seeker,realist,like_wa...",,beta_test,1,41,2022-10-18 21:32:04,Genesis
8,@latenightnft,imagination,visionary,"artist,seeker,shapeshifter,original,bonvivant,...","artist,seeker,shapeshifter,original,bonvivant,...",,beta_test,1,40,2022-10-18 21:32:04,Genesis
9,@musicalnetta,intellect,explorer,"believer,still,aequanimus,controller,realist,m...","believer,still,aequanimus,controller,realist,m...",,beta_test,1,39,2022-10-18 21:32:04,Genesis


In [8]:
## Get latest individuation for each tokenid based on timestamp

df = df.sort_values(by=["TokenID", "createdAt", "Token"], ascending=False).drop_duplicates(subset=["TokenID", "Token"], keep='first')

In [9]:
df

,q_1,q_2,q_3,q_4,q_5,address,nonce,getID,TokenID,createdAt,Token
id,,,,,,,,,,,
39,leemuel_eth,intellect,mentor,"enthusiast,believer,empathetic,moderator,optim...","enthusiast,believer,empathetic,moderator,optim...",,beta_test,1,48,2022-10-18 22:14:09,Genesis
1,@shaman4646,intuition,team_player,"enthusiast,optimistic,like_water,director,conn...","enthusiast,optimistic,like_water,director,conn...",,beta_test,1,47,2022-10-18 21:32:04,Genesis
2,@Koa_GM,imagination,founder,"empathetic,adventurer,like_water,artist,still,...","empathetic,adventurer,like_water,artist,still,...",,beta_test,1,46,2022-10-18 21:32:04,Genesis
3,@Lu_rolu,intuition,idealist,"traditionalist,controller,ingenious,enthusiast...","traditionalist,controller,ingenious,enthusiast...",,beta_test,1,45,2022-10-18 21:32:04,Genesis
4,NFTGuru2,intellect,mentor,"bonvivant,achiever,guided,adventurer,connector...","bonvivant,achiever,guided,adventurer,connector...",,beta_test,1,44,2022-10-18 21:32:04,Genesis
5,lima_eth,intellect,mentor,"bonvivant,like_water,champion,ally,moderator,m...","bonvivant,like_water,champion,ally,moderator,m...",,beta_test,1,43,2022-10-18 21:32:04,Genesis
6,@rp_rolu,imagination,visionary,"ally,artist,shapeshifter,connector,empathetic,...","ally,artist,shapeshifter,connector,empathetic,...",,beta_test,1,42,2022-10-18 21:32:04,Genesis
7,Vissyarts,intellect,explorer,"optimistic,shapeshifter,seeker,realist,like_wa...","optimistic,shapeshifter,seeker,realist,like_wa...",,beta_test,1,41,2022-10-18 21:32:04,Genesis
8,@latenightnft,imagination,visionary,"artist,seeker,shapeshifter,original,bonvivant,...","artist,seeker,shapeshifter,original,bonvivant,...",,beta_test,1,40,2022-10-18 21:32:04,Genesis


In [10]:
# Create dataframe with all 12 traits in sequence

dict_ind = {}
list_ind = []


for i in range(len(df)):
    dict_ind = {}
    dict_ind[1] = df.iloc[i]['q_5'].split(',')[0].replace(" ", "_").lower()
    dict_ind[2] = df.iloc[i]['q_5'].split(',')[1].replace(" ", "_").lower()
    dict_ind[3] = df.iloc[i]['q_5'].split(',')[2].replace(" ", "_").lower()
    dict_ind[4] = df.iloc[i]['q_5'].split(',')[3].replace(" ", "_").lower()
    dict_ind[5] = df.iloc[i]['q_5'].split(',')[4].replace(" ", "_").lower()
    dict_ind[6] = df.iloc[i]['q_5'].split(',')[5].replace(" ", "_").lower()
    dict_ind[7] = df.iloc[i]['q_5'].split(',')[6].replace(" ", "_").lower()
    dict_ind[8] = df.iloc[i]['q_5'].split(',')[7].replace(" ", "_").lower()
    dict_ind[9] = df.iloc[i]['q_5'].split(',')[8].replace(" ", "_").lower()
    dict_ind[10] = df.iloc[i]['q_5'].split(',')[9].replace(" ", "_").lower()
    dict_ind[11] = df.iloc[i]['q_5'].split(',')[10].replace(" ", "_").lower()
    dict_ind[12] = df.iloc[i]['q_5'].split(',')[11].replace(" ", "_").lower()
    dict_ind2 = {value:key for key, value in list(dict_ind.items())[0:8]}
    dict_ind2['Archetype'] = df.iloc[i]['q_3'].replace(" ", "_").lower().lower()
    dict_ind2['TokenID'] = df.iloc[i]['TokenID']
    dict_ind2['Type'] = df.iloc[i]['Token'].title()
    dict_ind2['twitter'] = df.iloc[i]['q_1']
    dict_ind2['ind_id'] = df.index[i]
    list_ind.append(dict_ind2)
    
    
ml_df = pd.DataFrame(list_ind)

In [11]:
# Check if any archetype or 2nd trait is not in the file:

arche_list = ["idealist", "leader", "founder", "angel", "team_player", "entertainer", "trailblazer", "grinder", "visionary", "rebel", "explorer", "mentor"]
sec_traits = pd.read_csv("Kinds_sec.csv") ## Attention to capital letters vs the files and sequence in form
all_traits = list(sec_traits["Name"]) + arche_list

In [12]:
# Create dummy variables for Archetype (people)

dummy = pd.get_dummies(ml_df["Archetype"])
dummy.columns = dummy.columns.str.lower()

ml_df_f = pd.concat([ml_df, dummy], axis=1)
#ml_df_f = pd.concat([ml_df, dummy], axis=1).set_index('ind_id')
ml_df_f = ml_df_f.drop(columns=["Archetype"])

In [13]:
all_traits_diff = set(all_traits).difference(ml_df_f.drop(columns=['twitter', 'TokenID', 'Type', 'ind_id']).columns)
add_traits = pd.DataFrame(columns=list(all_traits_diff))

In [14]:
ml_df_f = pd.concat([ml_df_f, add_traits], axis=1).fillna(0)

In [15]:
# Only until Guilherme updates the individuation table
ml_df_f = ml_df_f.replace(to_replace = ['Genesis', 'Lite'], value=[1, 2])

**BACK TO** [Summary](#Summary)

______________________________

## Check activation table

TO-DO: change to soul_form, add soul type when table is changed

In [2]:
## SQL connection

conn2=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor2 = conn2.cursor()

## Get individuation records

activation=pd.read_sql_query("SELECT * FROM XXX ",con=conn2)
#activation = activation.set_index('SOUL_ID')

## Close connection

conn2.close()

In [3]:
activation

,SOUL_ID,SOUL_TYPE_ID,CORD_TAG,CORD_ID,TWITTER_NAME,TWITTER_ID,CLUBS_ID,DATE_ACTIVATION,ACTIVE
0,245,1,_zr0#0001,457937292364087297,zr0art,1398108899576852480,None,2022-10-18 00:05:45,0
1,123,1,SuMusic#5636,582299913740877834,versusbysu,1393333043092406276,None,2022-10-18 00:16:28,0
2,247,1,_zr0#0001,457937292364087297,zr0art,1398108899576852480,None,2022-10-19 14:39:10,0


In [224]:
# Get latest activation for each soul_id

## add soul type when table is changed

activation = activation.sort_values(by=['SOUL_ID','DATE', 'SOUL_TYPE_ID'], ascending=False).drop_duplicates(subset=['SOUL_ID', 'SOUL_TYPE_ID'], keep='first')

In [334]:
ml_df_f

,player,connector,optimistic,like_water,director,believer,enthusiast,achiever,TokenID,Type,...,grinder,idealist,leader,mentor,rebel,team_player,trailblazer,visionary,entertainer,keeper
1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,41,1,...,0,0,0,0,0,0,1,0,0,0
2,6.0,5.0,2.0,3.0,4.0,0.0,1.0,0.0,38,1,...,0,0,0,0,0,1,0,0,0,0
3,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,37,1,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0.0,36,1,...,0,1,0,0,0,0,0,0,0,0
5,8.0,5.0,0.0,0.0,0.0,0.0,6.0,2.0,35,1,...,0,0,0,1,0,0,0,0,0,0
6,0.0,7.0,0.0,2.0,0.0,0.0,0.0,0.0,34,1,...,0,0,0,1,0,0,0,0,0,0
7,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,33,1,...,0,0,0,0,0,0,0,1,0,0
8,0.0,0.0,1.0,5.0,0.0,0.0,0.0,6.0,32,1,...,0,0,0,0,0,0,0,0,0,0
9,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,31,1,...,0,0,0,0,0,0,0,1,0,0
10,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,30,1,...,0,0,0,0,0,0,0,0,0,0


In [390]:
# Check if Soul id was activated

## add soul type when table is changed

ml_df_f_act = pd.merge(ml_df_f, activation, right_on=['TWITTER_NAME', 'SOUL_ID', 'SOUL_TYPE_ID'], left_on = ['twitter', 'TokenID', 'Type'], how='inner')
ml_df_f_act = ml_df_f_act.rename(columns={'SOUL_ID':'soul_id', 'SOUL_TYPE_ID':'soul_type'})
#ml_df_f_act = ml_df_f_act.set_index(['soul_id', 'soul_type']) ## add soul type to the index

In [391]:
ml_df_f_act = ml_df_f_act.drop(columns=['CORD_ID', 'CORD_TAG', 'DATE', 'TWITTER_NAME', 'TWITTER_ID', 'CLUBS_ID', 'WALLETS_VAULT_ID'])

In [392]:
ml_df_f_act

,player,connector,optimistic,like_water,director,believer,enthusiast,achiever,TokenID,Type,...,leader,mentor,rebel,team_player,trailblazer,visionary,entertainer,keeper,soul_id,soul_type
0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,41,1,...,0,0,0,0,1,0,0,0,41,1
1,6.0,5.0,2.0,3.0,4.0,0.0,1.0,0.0,38,1,...,0,0,0,1,0,0,0,0,38,1
2,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,37,1,...,0,0,0,0,0,0,0,0,37,1
3,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0.0,36,1,...,0,0,0,0,0,0,0,0,36,1


**BACK TO** [Summary](#Summary)

----------------------------------

## Add weights for matching

In [16]:
ml_df_f_act = ml_df_f.copy()

# Calculate similarity based on kind weights

politics_weight = [100, 100, 100]

spiritual_weight = [100, 100, 100]

other_weights = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]


# Adjust weights for political traits:

ml_df_f_act[['traditionalist', 'modernist']] = ml_df_f_act[['traditionalist', 'modernist']].replace(to_replace=[1], value=[80])


# Spiritual

ml_df_f_act[['agnostic', 'guided']] = ml_df_f_act[['agnostic', 'guided']].replace(to_replace=[1], value=[80])


# Adjust weights for main archetype (people)

ml_df_f_act[["idealist", "leader", "founder", "angel", "team_player", "entertainer", "trailblazer", "grinder", "visionary", "rebel", "explorer", "mentor"]] = ml_df_f_act[["idealist", "leader", "founder", "angel", "team_player", "entertainer", "trailblazer", "grinder", "visionary", "rebel", "explorer", "mentor"]].replace(to_replace=[1], value=[50])


# Adjust weights for other traits:

ml_df_f_act[['artist', 'connector', 'bonvivant', 'still', 'blindfolded', 'realist',
       'controller', 'aequanimus', 'nomad', 'player', 'poet', 'director',
       'like_water', 'optimistic', 'enthusiast', 'shapeshifter', 'jumper',
       'seeker', 'achiever', 'champion', 'juggler', 'keeper',
       'servant', 'original', 'empathetic', 'ingenious', 'ally',
       'perfectionist', 'adventurer', 'simulation', 'moderator', 'believer']] = ml_df_f_act[['artist', 'connector', 'bonvivant', 'still', 'blindfolded', 'realist',
       'controller', 'aequanimus', 'nomad', 'player', 'poet', 'director',
       'like_water', 'optimistic', 'enthusiast', 'shapeshifter', 'jumper',
       'seeker', 'achiever', 'champion', 'juggler', 'keeper',
       'servant', 'original', 'empathetic', 'ingenious', 'ally',
       'perfectionist', 'adventurer', 'simulation', 'moderator', 'believer']].replace(to_replace=[1], value=[40])

In [17]:
# if no political or religious selection, use default

index1 = ml_df_f_act.loc[(ml_df_f_act["guided"] == 0) & (ml_df_f_act["agnostic"] == 0) & (ml_df_f_act["believer"] == 0) & (ml_df_f_act["simulation"] == 0)].index

ml_df_f_act.loc[index1,["believer"]] = ml_df_f_act.loc[index1,["believer"]].replace(to_replace = [0], value=[20])

index2 = ml_df_f_act.loc[(ml_df_f_act["modernist"] == 0) & (ml_df_f_act["traditionalist"] == 0) & (ml_df_f_act["moderator"] == 0)].index

#for i in range(len(index2)):
#    ml_df_f_act.loc[index2[i],["moderator"]] = ml_df_f_act.loc[index2[i],["moderator"]].replace(to_replace = [0], value=[20])

ml_df_f_act.loc[index2,["moderator"]] = ml_df_f_act.loc[index2,["moderator"]].replace(to_replace = [0], value=[20])

In [18]:
ml_df_f_act

,enthusiast,believer,empathetic,moderator,optimistic,realist,controller,poet,TokenID,Type,...,grinder,idealist,leader,mentor,rebel,team_player,trailblazer,visionary,keeper,entertainer
0,40.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,48,1,...,0,0,0,50,0,0,0,0,0,0
1,40.0,20.0,0.0,20.0,2.0,0.0,0.0,0.0,47,1,...,0,0,0,0,0,50,0,0,0,0
2,0.0,0.0,40.0,20.0,6.0,0.0,0.0,0.0,46,1,...,0,0,0,0,0,0,0,0,0,0
3,4.0,7.0,0.0,0.0,0.0,0.0,2.0,5.0,45,1,...,0,50,0,0,0,0,0,0,0,0
4,6.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,44,1,...,0,0,0,50,0,0,0,0,0,0
5,0.0,20.0,8.0,5.0,0.0,0.0,0.0,0.0,43,1,...,0,0,0,50,0,0,0,0,0,0
6,0.0,20.0,5.0,20.0,8.0,0.0,0.0,0.0,42,1,...,0,0,0,0,0,0,0,50,0,0
7,0.0,20.0,0.0,8.0,40.0,4.0,0.0,0.0,41,1,...,0,0,0,0,0,0,0,0,0,0
8,0.0,20.0,0.0,20.0,0.0,0.0,0.0,0.0,40,1,...,0,0,0,0,0,0,0,50,0,0
9,0.0,40.0,0.0,0.0,0.0,5.0,4.0,0.0,39,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# New vectors based on archetype circle


## 1st level

#ml_df_f_act["ideal_founder"] = (ml_df_f_act["Idealist"] + ml_df_f_act["Founder"])*0.5
#ml_df_f_act["founder_leader"] = (ml_df_f_act["Founder"] + ml_df_f_act["Leader"])*0.5
#ml_df_f_act["leader_angel"] = (ml_df_f_act["Leader"] + ml_df_f_act["Angel"])*0.5
#ml_df_f_act["angel_team"] = (ml_df_f_act["Angel"] + ml_df_f_act["Team Player"])*0.5
#ml_df_f_act["team_entertainer"] = (ml_df_f_act["Team Player"] + ml_df_f_act["Entertainer"])*0.5
#ml_df_f_act["entertainer_trail"] = (ml_df_f_act["Entertainer"] + ml_df_f_act["Trailblazer"])*0.5
#ml_df_f_act["trail_grinder"] = (ml_df_f_act["Trailblazer"] + ml_df_f_act["Grinder"])*0.5
#ml_df_f_act["grinder_visionary"] = (ml_df_f_act["Grinder"] + ml_df_f_act["Visionary"])*0.5
#ml_df_f_act["visionary_rebel"] = (ml_df_f_act["Visionary"] + ml_df_f_act["Rebel"])*0.5
#ml_df_f_act["rebel_explorer"] = (ml_df_f_act["Rebel"] + ml_df_f_act["Explorer"])*0.5
#ml_df_f_act["explorer_mentor"] = (ml_df_f_act["Explorer"] + ml_df_f_act["Mentor"])*0.5
#ml_df_f_act["mentor_ideal"] = (ml_df_f_act["Mentor"] + ml_df_f_act["Idealist"])*0.5


## 2nd level

ml_df_f_act["ideal_leader"] = (ml_df_f_act["idealist"] + ml_df_f_act["leader"])
ml_df_f_act["founder_angel"] = (ml_df_f_act["founder"] + ml_df_f_act["angel"])
ml_df_f_act["leader_team"] = (ml_df_f_act["leader"] + ml_df_f_act["team_player"])
ml_df_f_act["angel_entertainer"] = (ml_df_f_act["angel"] + ml_df_f_act["entertainer"])
ml_df_f_act["team_trail"] = (ml_df_f_act["team_player"] + ml_df_f_act["trailblazer"])
ml_df_f_act["entertainer_grinder"] = (ml_df_f_act["entertainer"] + ml_df_f_act["grinder"])
ml_df_f_act["trail_visionary"] = (ml_df_f_act["trailblazer"] + ml_df_f_act["visionary"])
ml_df_f_act["grinder_rebel"] = (ml_df_f_act["grinder"] + ml_df_f_act["rebel"])
ml_df_f_act["visionary_explorer"] = (ml_df_f_act["visionary"] + ml_df_f_act["explorer"])
ml_df_f_act["rebel_mentor"] = (ml_df_f_act["rebel"] + ml_df_f_act["mentor"])
ml_df_f_act["explorer_ideal"] = (ml_df_f_act["explorer"] + ml_df_f_act["idealist"])
ml_df_f_act["mentor_founder"] = (ml_df_f_act["mentor"] + ml_df_f_act["founder"])


In [20]:
# drop single archetypes if using archetype combinations

ml_df_f_act = ml_df_f_act.drop(["idealist", "leader", "founder", "angel", "team_player", "entertainer", "trailblazer", "grinder", "visionary", "rebel", "explorer", "mentor", "TokenID", "Type"], axis=1)

In [22]:
# re-order columns

ml_df_f_act = ml_df_f_act[['bonvivant', 'blindfolded', 'like_water', 'agnostic', 'traditionalist',
       'nomad', 'modernist', 'empathetic', 'realist', 'simulation',
       'connector', 'artist', 'controller', 'aequanimus', 'guided', 'servant',
       'original', 'believer', 'player', 'poet', 'director', 'still', 'jumper',
       'moderator', 'juggler', 'keeper', 'champion', 'optimistic',
       'shapeshifter', 'seeker', 'ingenious', 'achiever', 'adventurer',
       'enthusiast', 'perfectionist', 'ally', 'ideal_leader', 'founder_angel',
       'leader_team', 'angel_entertainer', 'team_trail', 'entertainer_grinder',
       'trail_visionary', 'grinder_rebel', 'visionary_explorer',
       'rebel_mentor', 'explorer_ideal', 'mentor_founder', 'twitter', 'ind_id']]



In [23]:
ml_df_f_act = ml_df_f_act.fillna(0)

In [24]:
ml_df_f_act

,bonvivant,blindfolded,like_water,agnostic,traditionalist,nomad,modernist,empathetic,realist,simulation,...,team_trail,entertainer_grinder,trail_visionary,grinder_rebel,visionary_explorer,rebel_mentor,explorer_ideal,mentor_founder,twitter,ind_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.0,...,0,0,0,0,0,50,0,50,leemuel_eth,39
1,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,50,0,0,0,0,0,0,0,@shaman4646,1
2,0.0,0.0,3.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,...,0,0,0,0,0,0,0,50,@Koa_GM,2
3,0.0,0.0,0.0,0.0,80.0,0.0,6.0,0.0,0.0,0.0,...,0,0,0,0,0,0,50,0,@Lu_rolu,3
4,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,50,0,50,NFTGuru2,4
5,40.0,0.0,2.0,0.0,0.0,0.0,6.0,8.0,0.0,0.0,...,0,0,0,0,0,50,0,50,lima_eth,5
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0,0,50,0,50,0,0,0,@rp_rolu,6
7,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0,0,0,0,50,0,50,0,Vissyarts,7
8,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,50,0,50,0,0,0,@latenightnft,8
9,0.0,0.0,8.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,...,0,0,0,0,50,0,50,0,@musicalnetta,9


In [25]:
ml_df_f_act = ml_df_f_act.set_index('ind_id')

In [26]:
ml_df_f_act

,bonvivant,blindfolded,like_water,agnostic,traditionalist,nomad,modernist,empathetic,realist,simulation,...,angel_entertainer,team_trail,entertainer_grinder,trail_visionary,grinder_rebel,visionary_explorer,rebel_mentor,explorer_ideal,mentor_founder,twitter
ind_id,,,,,,,,,,,,,,,,,,,,,
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.0,...,0,0,0,0,0,0,50,0,50,leemuel_eth
1,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,50,0,0,0,0,0,0,0,@shaman4646
2,0.0,0.0,3.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,...,0,0,0,0,0,0,0,0,50,@Koa_GM
3,0.0,0.0,0.0,0.0,80.0,0.0,6.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,50,0,@Lu_rolu
4,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,50,0,50,NFTGuru2
5,40.0,0.0,2.0,0.0,0.0,0.0,6.0,8.0,0.0,0.0,...,0,0,0,0,0,0,50,0,50,lima_eth
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0,0,0,50,0,50,0,0,0,@rp_rolu
7,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0,0,0,0,0,50,0,50,0,Vissyarts
8,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,50,0,50,0,0,0,@latenightnft


In [27]:
# Normalize vector elements

from sklearn import preprocessing
import numpy as np

vectors_norm = preprocessing.normalize(ml_df_f_act.drop(columns=['twitter']))

In [28]:
vectors_norm

array([[0.        , 0.        , 0.        , ..., 0.60620536, 0.        ,
        0.60620536],
       [0.        , 0.        , 0.03440557, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.03534798, ..., 0.        , 0.        ,
        0.58913293],
       ...,
       [0.        , 0.        , 0.08486875, ..., 0.        , 0.        ,
        0.        ],
       [0.04713063, 0.05891329, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08027966, 0.        , 0.        , ..., 0.57342617, 0.        ,
        0.        ]])

In [39]:
# Reorganizing vector dataframe with columns and index that will be used later

vector_df = pd.DataFrame(vectors_norm, columns=ml_df_f_act.drop(columns=['twitter']).columns)
#vector_df['soul_id'] = ml_df_f_act['soul_id']
#vector_df['soul_type'] = ml_df_f_act['soul_type']
vector_df['twitter'] = ml_df_f_act['twitter']
#vector_df = pd.merge(vector_df, ml_df_f_act[['twitter', 'soul_id', 'soul_type']], left_on=['soul_id', 'soul_type'], right_on=['soul_id', 'soul_type'], how='inner')

In [40]:
vector_df['ind_id'] = ml_df_f_act.index

In [41]:
vector_df

,bonvivant,blindfolded,like_water,agnostic,traditionalist,nomad,modernist,empathetic,realist,simulation,...,team_trail,entertainer_grinder,trail_visionary,grinder_rebel,visionary_explorer,rebel_mentor,explorer_ideal,mentor_founder,twitter,ind_id
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036372,0.072745,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.606205,0.000000,0.606205,NaN,39
1,0.000000,0.000000,0.034406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.573426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,@shaman4646,1
2,0.000000,0.000000,0.035348,0.000000,0.000000,0.000000,0.000000,0.471306,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,@Koa_GM,2
3,0.000000,0.000000,0.000000,0.000000,0.742685,0.000000,0.055701,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.464178,0.000000,@Lu_rolu,3
4,0.471306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,NFTGuru2,4
5,0.471306,0.000000,0.023565,0.000000,0.000000,0.000000,0.070696,0.094261,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,lima_eth,5
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057343,0.000000,0.000000,...,0.000000,0.000000,0.573426,0.000000,0.573426,0.000000,0.000000,0.000000,@rp_rolu,6
7,0.000000,0.000000,0.058913,0.000000,0.000000,0.000000,0.000000,0.000000,0.047131,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,0.000000,Vissyarts,7
8,0.057343,0.091748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.573426,0.000000,0.573426,0.000000,0.000000,0.000000,@latenightnft,8
9,0.000000,0.000000,0.096993,0.000000,0.000000,0.000000,0.072745,0.000000,0.060621,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.606205,0.000000,0.606205,0.000000,@musicalnetta,9


**BACK TO** [Summary](#Summary)

## Run matching

n neighbors need to be adjusted as we get more people

In [277]:
## SQL connection

conn2=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor2 = conn2.cursor()

## Get individuation records

matches_table=pd.read_sql_query("SELECT * FROM `XXX`",con=conn2) 

## Close connection

conn2.close()

In [278]:
matches_table

,soul_req,soul_match,match_gen_time,match_access_time,review,review_time,flag,soul_req_type,soul_match_type
0,22,7.0,2022-09-04 20:39:56,0000-00-00 00:00:00,good,0000-00-00 00:00:00,,1.0,1.0
1,14,22.0,2022-09-04 20:39:56,0000-00-00 00:00:00,bad,0000-00-00 00:00:00,,1.0,1.0
2,7,22.0,2022-09-04 20:39:56,0000-00-00 00:00:00,None,0000-00-00 00:00:00,,1.0,1.0
3,6,4.0,2022-09-04 20:39:56,0000-00-00 00:00:00,None,0000-00-00 00:00:00,,1.0,1.0
4,5,1.0,2022-09-04 20:39:56,0000-00-00 00:00:00,None,0000-00-00 00:00:00,,1.0,1.0
5,4,6.0,2022-09-04 20:39:56,0000-00-00 00:00:00,None,0000-00-00 00:00:00,,1.0,1.0
6,1,5.0,2022-09-04 20:39:56,0000-00-00 00:00:00,not found,0000-00-00 00:00:00,,1.0,1.0
7,38,NaN,2022-09-05 23:15:04,0000-00-00 00:00:00,None,0000-00-00 00:00:00,X,NaN,NaN
8,37,NaN,2022-09-05 23:15:04,0000-00-00 00:00:00,None,0000-00-00 00:00:00,X,NaN,NaN
9,36,NaN,2022-09-05 23:15:04,0000-00-00 00:00:00,None,0000-00-00 00:00:00,X,NaN,NaN


In [286]:
# Check if n_neighbors is big enough to cover the most active user:

max_match = matches_table.groupby(['soul_req', 'soul_req_type']).count()[['soul_match', 'soul_match_type']].max()

In [287]:
max_match

soul_match         1
soul_match_type    2
dtype: int64

**>>>>>>>>> n_neigh needs to be higher than max_match**

In [91]:
n_individuation = vector_df.shape[1]
n_individuation

50

In [42]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = vector_df.drop(columns=['twitter', 'ind_id'])
#samples=vectors_norm

n_neigh = 4 ######## higher than max_match

neigh = NearestNeighbors(n_neighbors=n_neigh, algorithm='brute', metric='cosine').fit(samples)
dist, ind = neigh.kneighbors(samples)



In [43]:
matches_id = pd.DataFrame(ind)

In [44]:
dist

array([[0.        , 0.21744461, 0.23677867, 0.24001558],
       [0.        , 0.23556491, 0.51515465, 0.5567746 ],
       [0.        , 0.23268083, 0.26786731, 0.26813757],
       [0.        , 0.44869912, 0.44935543, 0.44976924],
       [0.        , 0.06205748, 0.23138229, 0.24001558],
       [0.        , 0.06205748, 0.21732881, 0.26641677],
       [0.        , 0.22228068, 0.22705814, 0.23661712],
       [0.        , 0.16087434, 0.44213602, 0.53420794],
       [0.        , 0.21489647, 0.22228068, 0.22662107],
       [0.        , 0.16087434, 0.43182259, 0.52643828],
       [0.        , 0.2213827 , 0.24892406, 0.54505067],
       [0.        , 0.22662107, 0.23661712, 0.24543578],
       [0.        , 0.21922062, 0.23268083, 0.25084097],
       [0.        , 0.39225641, 0.43249792, 0.43557291],
       [0.        , 0.21922062, 0.21978167, 0.22692301],
       [0.        , 0.21143968, 0.21744461, 0.25031237],
       [0.        , 0.01957518, 0.22692301, 0.23340924],
       [0.        , 0.2213827 ,

In [45]:
# Create and organize df for match info

matches_dist = pd.DataFrame(dist).reset_index()

In [46]:
col = list(range(n_neigh))

In [47]:
vector_df

,bonvivant,blindfolded,like_water,agnostic,traditionalist,nomad,modernist,empathetic,realist,simulation,...,team_trail,entertainer_grinder,trail_visionary,grinder_rebel,visionary_explorer,rebel_mentor,explorer_ideal,mentor_founder,twitter,ind_id
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036372,0.072745,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.606205,0.000000,0.606205,NaN,39
1,0.000000,0.000000,0.034406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.573426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,@shaman4646,1
2,0.000000,0.000000,0.035348,0.000000,0.000000,0.000000,0.000000,0.471306,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,@Koa_GM,2
3,0.000000,0.000000,0.000000,0.000000,0.742685,0.000000,0.055701,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.464178,0.000000,@Lu_rolu,3
4,0.471306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,NFTGuru2,4
5,0.471306,0.000000,0.023565,0.000000,0.000000,0.000000,0.070696,0.094261,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,lima_eth,5
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057343,0.000000,0.000000,...,0.000000,0.000000,0.573426,0.000000,0.573426,0.000000,0.000000,0.000000,@rp_rolu,6
7,0.000000,0.000000,0.058913,0.000000,0.000000,0.000000,0.000000,0.000000,0.047131,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.589133,0.000000,0.589133,0.000000,Vissyarts,7
8,0.057343,0.091748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.573426,0.000000,0.573426,0.000000,0.000000,0.000000,@latenightnft,8
9,0.000000,0.000000,0.096993,0.000000,0.000000,0.000000,0.072745,0.000000,0.060621,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.606205,0.000000,0.606205,0.000000,@musicalnetta,9


In [48]:
matches_dist[['index']] = matches_dist[['index']].replace(to_replace=list(vector_df.index), value=list(vector_df['ind_id']))
#matches_dist[['soul_type']] = vector_df['soul_type']
matches_id[col] = matches_id[col].replace(to_replace=list(vector_df.index), value=list(vector_df['ind_id']))

In [49]:
matches_id

,0,1,2,3
0,39,15,35,22
1,1,28,32,24
2,2,12,34,25
3,3,16,18,26
4,4,5,35,39
5,5,4,35,15
6,6,8,33,11
7,7,9,23,14
8,8,33,6,11
9,9,7,23,19


In [299]:
#matches_dist['soul_type'] = vector_df['soul_type']

In [50]:
matches_dist = matches_dist.rename(columns={'index': 'ind_id'})

In [51]:
matches_id

,0,1,2,3
0,39,15,35,22
1,1,28,32,24
2,2,12,34,25
3,3,16,18,26
4,4,5,35,39
5,5,4,35,15
6,6,8,33,11
7,7,9,23,14
8,8,33,6,11
9,9,7,23,19


In [52]:
matches_id_melt = pd.melt(matches_id, id_vars=0, var_name='matches_seq', value_name='match_ind_id').sort_values([0, 'matches_seq'])

In [53]:
matches_id_melt

,0,matches_seq,match_ind_id
1,1,1,28
40,1,2,32
79,1,3,24
2,2,1,12
41,2,2,34
...,...,...,...
77,38,2,15
116,38,3,5
0,39,1,15
39,39,2,35


In [54]:
matches_dist_melt = pd.melt(matches_dist.drop(columns=0), id_vars='ind_id', var_name='matches_seq_2', value_name='match_dist').sort_values(['ind_id', 'matches_seq_2'])

In [55]:
matches_dist

,ind_id,0,1,2,3
0,39,0.0,0.217445,0.236779,0.240016
1,1,0.0,0.235565,0.515155,0.556775
2,2,0.0,0.232681,0.267867,0.268138
3,3,0.0,0.448699,0.449355,0.449769
4,4,0.0,0.062057,0.231382,0.240016
5,5,0.0,0.062057,0.217329,0.266417
6,6,0.0,0.222281,0.227058,0.236617
7,7,0.0,0.160874,0.442136,0.534208
8,8,0.0,0.214896,0.222281,0.226621
9,9,0.0,0.160874,0.431823,0.526438


In [57]:
matches = pd.merge(matches_id_melt, matches_dist_melt, left_on=[0, 'matches_seq'], right_on=['ind_id', 'matches_seq_2'], how='inner')   

In [61]:
matches

,matches_seq,match_ind_id,ind_id,match_dist
0,1,28,1,0.235565
1,2,32,1,0.515155
2,3,24,1,0.556775
3,1,12,2,0.232681
4,2,34,2,0.267867
...,...,...,...,...
112,2,15,38,0.522722
113,3,5,38,0.548532
114,1,15,39,0.217445
115,2,35,39,0.236779


In [58]:
matches = matches.drop(columns=[0, 'matches_seq_2'])

In [62]:
# Reorder columns 

matches = matches[['ind_id', 'matches_seq', 'match_ind_id', 'match_dist']]

In [63]:
# Change column names

matches.columns = ['soul_req', 'matches_seq', 'soul_match', 'dist']

In [64]:
matches['score'] = 1 - matches['dist']

/Users/tpancier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
matches.sort_values('score', ascending=False)

,soul_req,matches_seq,soul_match,dist,score
45,16,1,18,0.019575,0.980425
51,18,1,16,0.019575,0.980425
9,4,1,5,0.062057,0.937943
12,5,1,4,0.062057,0.937943
105,36,1,31,0.073678,0.926322
...,...,...,...,...,...
89,30,3,28,0.557315,0.442685
80,27,3,34,0.582584,0.417416
62,21,3,29,0.595169,0.404831
107,36,3,29,0.642864,0.357136


**BACK TO** [Summary](#Summary)

## Confirm matches to send

Scenarios:
- no access time -> user didn't check last match -> don't send new match (included in no review info)
- no review info -> user didn't review last match  -> don't send new match
- send new matches everyday if review is done for the last one. (look gen_time)
- if review is: not able to talk, send people again after a while (to be implemented)
- remove matches that have a review

In [92]:
## SQL connection

conn2=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor2 = conn2.cursor()

## Get individuation records

matches_table=pd.read_sql_query("SELECT * FROM `XXX`",con=conn2) 

## Close connection

conn2.close()

In [93]:
# Get last match

matches_check1 = matches_table.sort_values(by=['soul_req', 'match_gen_time'], ascending=False).drop_duplicates(subset="soul_req", keep='first')

In [94]:
# Check last match was reviewed/accessed. No new match if previous was not reviewed. 

souls_to_drop1 = matches_check1[matches_check1['review'].isna()]['soul_req'].values

In [105]:
# Check if a match was already provided today


#matches_table_access = matches_table[matches_table['match_access_time'].notna()][['match_access_time']]
matches_check1.loc[:,'match_gen_time'] = matches_check1['match_gen_time'].dt.tz_localize('UTC')

d = datetime.now(timezone.utc) - timedelta(days=0)  ## timedelta can be changed if not doing every day

souls_to_drop2 = matches_check1[matches_check1['match_gen_time'] > d]['soul_req'].values

In [107]:
souls_to_drop = list(set(list(souls_to_drop1) + list(souls_to_drop2)))

In [108]:
# Drop souls from match

matches_final = matches.drop(matches[matches['soul_req'].isin(souls_to_drop)].index)
matches_final

,soul_req,matches_seq,soul_match,dist,score
0,0,1,2,0.259489,0.740511
1,0,2,1,0.558727,0.441273
2,0,3,3,0.849928,0.150072
3,0,4,5,0.855846,0.144154
4,0,5,4,0.887095,0.112905
5,1,1,0,0.558727,0.441273
6,1,2,2,0.591908,0.408092
7,1,3,4,0.861928,0.138072
8,1,4,5,0.872156,0.127844
9,1,5,3,0.882678,0.117322


In [642]:
# Select matches with review

matches_to_drop3 = matches_check1[matches_check1['review'].notna()][['soul_req', 'soul_match']]

# drop from matches the combination of soul_req and soul_match in matches_to_drop3 (both ways)

matches_final = matches_final.drop(matches_final[matches_final[['soul_req', 'soul_match']].apply(tuple, axis=1).isin(zip(list(matches_to_drop3['soul_req']), list(matches_to_drop3['soul_match'])))].index, axis=0)
matches_final = matches_final.drop(matches_final[matches_final[['soul_req', 'soul_match']].apply(tuple, axis=1).isin(zip(list(matches_to_drop3['soul_match']), list(matches_to_drop3['soul_req'])))].index, axis=0)



In [643]:
# get only the best match available

matches_final = matches_final.sort_values(by=['soul_req', 'score'], ascending=False).drop_duplicates(subset="soul_req", keep='first')

In [644]:
# Organize table in database format

matches_final = matches_final.drop(columns=['matches_seq', 'dist', 'score'])
matches_final['match_gen_time'] = datetime.now(timezone.utc)
matches_final['flag'] = 'null'

In [109]:
# Check twitter discrepancy and add flag if any

indiv_twitter = ml_df_f.loc[ml_df_f.index, 'twitter']
twitter = pd.merge(indiv_twitter, activation, left_on=indiv_twitter.index, right_on='SOUL_ID', how='left')


### in the future, maybe filter by date from the individuation table and from the activation as the trigger

In [111]:
flag_table = twitter[twitter['twitter'] != twitter['TWITTER_NAME']]

In [113]:
# Check if there's no flag already as last record in match table



In [519]:
matches_final_flag = pd.DataFrame()
matches_final_flag['soul_req'] = flag_table['SOUL_ID']
matches_final_flag['match_gen_time'] = datetime.now(timezone.utc)
matches_final_flag['flag'] = 'X'

In [525]:
matches_final = pd.concat([matches_final, matches_final_flag], axis=0)

In [524]:

## SQL connection

db_data = 'XXX' + ':' + 'XXX' + '@' + 'XXX' + ':3306/' + 'XXX' + 'XXX'
engine = create_engine(db_data)

conn2=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor2 = conn2.cursor()


# Execute the to_sql for writting DF into SQL
matches_final.to_sql('XXX', engine, if_exists='append', index=False)    


# Execute query
sql = "SELECT * FROM `XXX`"
cursor2.execute(sql)


engine.dispose()
conn2.close()


**BACK TO** [Summary](#Summary)

## Delete records in match table

In [340]:
## just for test, records deleted from table
## SQL connection

conn2=pymysql.connect(host='XXX', password='XXX',port=int(3306),user='XXX',db='XXX')
cursor2 = conn2.cursor()


cursor2.execute("DELETE FROM `XXX`")


conn2.commit()

## Close connection

conn2.close()

## Reintroduce review 'not found' (after a while)

**BACK TO** [Summary](#Summary)